In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [ ]:
mimic_iv_path = "/cis/home/charr165/Documents/physionet.org/mimiciv/2.2"
mm_dir = "/cis/home/charr165/Documents/multimodal"

output_dir = os.path.join(mm_dir, "preprocessing")

In [ ]:
include_notes = True
include_cxr = True
standard_scale = True
include_missing = False

In [ ]:
# ireg_vitals_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_vitals_icu.pkl"))
# imputed_vitals = pd.read_pickle(os.path.join(output_dir, "imputed_ts_vitals_icu.pkl"))

ireg_vitals_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_labs_vitals_icu.pkl"))
imputed_vitals = pd.read_pickle(os.path.join(output_dir, "imputed_ts_labs_vitals_icu.pkl"))

ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['timedelta'] >= 0]
imputed_vitals = imputed_vitals[imputed_vitals['timedelta'] >= 0]

In [ ]:
if include_notes:
    notes_df = pd.read_pickle(os.path.join(output_dir, "icu_notes_text_embeddings.pkl"))
    # notes_df = pd.read_pickle(os.path.join(output_dir, "notes_text.pkl"))
    notes_df = notes_df[notes_df['stay_id'].notnull()]

    notes_df = notes_df[notes_df['icu_time_delta'] >= 0]

if include_cxr:
    cxr_df = pd.read_pickle(os.path.join(output_dir, "cxr_embeddings_icu.pkl"))
    cxr_df = cxr_df[cxr_df['icu_time_delta'] >= 0]

In [ ]:
icustays_df = pd.read_csv(os.path.join(mimic_iv_path, "icu", "icustays.csv"), low_memory=False)
icustays_df['intime'] = pd.to_datetime(icustays_df['intime'])
icustays_df['outtime'] = pd.to_datetime(icustays_df['outtime'])

In [ ]:
valid_stay_ids = icustays_df['stay_id'].unique()

ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'].isin(valid_stay_ids)]
imputed_vitals = imputed_vitals[imputed_vitals['stay_id'].isin(valid_stay_ids)]

if include_notes:
    notes_df = notes_df[notes_df['stay_id'].isin(valid_stay_ids)]

if include_cxr:
    cxr_df = cxr_df[cxr_df['stay_id'].isin(valid_stay_ids)]

In [ ]:
admissions_df = pd.read_csv(os.path.join(mimic_iv_path, "hosp", "admissions.csv"))
admissions_df = admissions_df.rename(columns={"hospital_expire_flag": "died"})
admissions_df = admissions_df[["subject_id", "admittime", "dischtime", "hadm_id", "died"]]
admissions_df['admittime'] = pd.to_datetime(admissions_df['admittime'])
admissions_df['dischtime'] = pd.to_datetime(admissions_df['dischtime'])

In [ ]:

if not include_missing:
    unique_stays = ireg_vitals_ts_df['stay_id'].unique()
    print(f"Number of stays with vitals: {len(unique_stays)}")

    if include_notes:
        unique_stays = np.intersect1d(unique_stays, notes_df['stay_id'].unique())
        print(f"Number of stays with notes: {len(unique_stays)}")

    if include_cxr:
        unique_stays = np.intersect1d(unique_stays, cxr_df['stay_id'].unique())
        print(f"Number of stays with cxr: {len(unique_stays)}")
else:
    unique_stays = ireg_vitals_ts_df['stay_id'].unique()
    print(f"Number of stays with vitals: {len(unique_stays)}")

    if include_notes:
        # Get stays with either TS or notes
        unique_stays = np.union1d(unique_stays, notes_df['stay_id'].unique())
        print(f"Number of stays with either TS or notes: {len(unique_stays)}")
    
    if include_cxr:
        unique_stays = np.union1d(unique_stays, cxr_df['stay_id'].unique())
        print(f"Number of stays with either TS, notes, cxr: {len(unique_stays)}")

In [ ]:
# Create train, val, test splits
np.random.seed(0)
np.random.shuffle(unique_stays)
train_stays = unique_stays[:int(0.7*len(unique_stays))]
val_stays = unique_stays[int(0.7*len(unique_stays)):int(0.85*len(unique_stays))]
test_stays = unique_stays[int(0.85*len(unique_stays)):]

In [ ]:
train_ireg_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'].isin(train_stays)].copy()
train_imputed_df = imputed_vitals[imputed_vitals['stay_id'].isin(train_stays)].copy()

cols = train_ireg_ts_df.columns.tolist()
cols = [col for col in cols if col not in ['subject_id', 'hadm_id', 'stay_id', 'timedelta']]

if standard_scale:
    for col in cols:
        scaler = StandardScaler()
        scaler.fit(train_ireg_ts_df[[col]])
        ireg_vitals_ts_df[col] = scaler.transform(ireg_vitals_ts_df[[col]])

        scaler = StandardScaler()
        scaler.fit(train_imputed_df[[col]])
        imputed_vitals[col] = scaler.transform(imputed_vitals[[col]])


In [ ]:
start_hr = 4
def get_stay_list(stays):
    stays_list = []

    for curr_stay in tqdm(stays):

        curr_stay_ireg_all = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'] == curr_stay].copy()

        max_time = int(np.max(curr_stay_ireg_all['timedelta']))

        if max_time < start_hr:
            continue

        for hr in np.arange(start_hr, max_time + 1):
            curr_stay_ireg = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'] == curr_stay].copy()
            curr_stay_imputed = imputed_vitals[imputed_vitals['stay_id'] == curr_stay].copy()

            curr_stay_ireg = curr_stay_ireg[curr_stay_ireg['timedelta'] <= hr]
            curr_stay_imputed = curr_stay_imputed[curr_stay_imputed['timedelta'] <= hr]

            if include_notes:
                curr_stay_notes = notes_df[notes_df['stay_id'] == curr_stay].copy()
                curr_stay_notes = curr_stay_notes[curr_stay_notes['icu_time_delta'] <= hr]

            if include_cxr:
                curr_stay_cxr = cxr_df[cxr_df['stay_id'] == curr_stay].copy()
                curr_stay_cxr = curr_stay_cxr[curr_stay_cxr['icu_time_delta'] <= hr]

            if len(curr_stay_ireg) == 0:
                continue

            if (len(curr_stay_notes) == 0) and not include_missing:
                continue
                
            if (len(curr_stay_cxr) == 0) and not include_missing:
                continue

            curr_admit_df = admissions_df[admissions_df['hadm_id'] == curr_stay_ireg['hadm_id'].iloc[0]].iloc[0].copy()
            curr_icu_df = icustays_df[icustays_df['stay_id'] == curr_stay_ireg['stay_id'].iloc[0]].iloc[0].copy()

            curr_time_to_end = (curr_icu_df['outtime'] - (curr_icu_df['intime'] + pd.Timedelta(hours=hr)))
            time_to_end = curr_time_to_end.total_seconds() / 3600
            # print(f"({hr}, {max_time}, {time_to_end}, {curr_admit_df['died']})", end="  ")
            if (time_to_end <= 24) and (curr_admit_df['died'] == 1):
                label = 1
            else:
                label = 0
            
            curr_stay_dict = {}
            curr_stay_dict['name'] = curr_stay_ireg['subject_id'].iloc[0]
            curr_stay_dict['hadm_id'] = curr_stay_ireg['hadm_id'].iloc[0]
            curr_stay_dict['stay_id'] = curr_stay
            curr_stay_dict['ts_tt'] = curr_stay_ireg['timedelta'].values

            curr_stay_ireg.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'timedelta'], inplace=True)
            ireg_ts_mask = curr_stay_ireg.notnull()
            curr_stay_ireg.fillna(0, inplace=True)
            curr_stay_dict['irg_ts'] = curr_stay_ireg.values
            curr_stay_dict['irg_ts_mask'] = ireg_ts_mask.values.astype(int)

            curr_stay_imputed.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'timedelta'], inplace=True)
            curr_stay_dict['reg_ts'] = curr_stay_imputed.values

            if include_notes:
                if len(curr_stay_notes) == 0:
                    curr_stay_dict['text_data'] = []
                    curr_stay_dict['text_time_to_end'] = []
                    curr_stay_dict['text_embeddings'] = []
                    curr_stay_dict['text_missing'] = 1
                else:
                    curr_stay_dict['text_data'] = curr_stay_notes['text'].tolist()
                    curr_stay_dict['text_time_to_end'] = curr_stay_notes['icu_time_delta'].values
                    curr_stay_dict['text_embeddings'] = [emb[0][0] for emb in curr_stay_notes['biobert_embeddings']]
                    curr_stay_dict['text_missing'] = 0

            if include_cxr:
                if len(curr_stay_cxr) == 0:
                    curr_stay_dict['cxr_feats'] = []
                    curr_stay_dict['cxr_time'] = []
                    curr_stay_dict['cxr_missing'] = 1
                else:
                    curr_stay_dict['cxr_feats'] = curr_stay_cxr['densefeatures'].tolist()
                    curr_stay_dict['cxr_time'] = curr_stay_cxr['icu_time_delta'].values
                    curr_stay_dict['cxr_missing'] = 0
            
            curr_stay_dict['label'] = label

            stays_list.append(curr_stay_dict)

        # print('\n')

    return stays_list

train_stays_list = get_stay_list(train_stays)
val_stays_list = get_stay_list(val_stays)
test_stays_list = get_stay_list(test_stays)

In [ ]:
n_notes = []
for item in train_stays_list:
    n_notes.append(len(item['text_data']))


In [ ]:
# Save the data
import pickle

base_name = "decompensation"
if restrict_48_hours:
    base_name += "-48"
else:
    base_name += "-all"

if include_cxr:
    if include_notes:
        base_name += "-cxr-notes"
    else:
        base_name += "-cxr"

if include_missing:
    base_name += "-missingInd"

f_path = os.path.join(output_dir, f"train_{base_name}_stays.pkl")
with open(f_path, 'wb') as f:
    print(f"Saving train stays to {f_path}")
    pickle.dump(train_stays_list, f)

f_path = os.path.join(output_dir, f"val_{base_name}_stays.pkl")
with open(f_path, 'wb') as f:
    print(f"Saving val stays to {f_path}")
    pickle.dump(val_stays_list, f)

f_path = os.path.join(output_dir, f"test_{base_name}_stays.pkl")
with open(f_path, 'wb') as f:
    print(f"Saving test stays to {f_path}")
    pickle.dump(test_stays_list, f)


In [ ]:
import pickle
import os

f_name = os.path.join(output_dir, "train_ihm-cxr_stays.pkl")

# Read in the pickled data
with open(f_name, 'rb') as f:
    train_stays_list = pickle.load(f)

In [ ]:
train_stays_list[4]['ts_tt'].max()